# LATAM Reto Ingeniero de Datos - Farmers protest tweets

## Reto
El reto consiste en responder tres preguntas claves para el análisis de tweets sobre "farmers protest". Este reto permite ir más allá y dejar volar la imaginación de forma que permita conocer los conocimientos y habilidades de los aspirantes.

## Arquitectura

Al ver el reto, el cual posee un archivo compreso cargado en un Google Drive público y teniendo en cuenta el tiempo de desarrollo, la necesidad y las restricciones por el tipo de suscripción en el servicio de la nube; se planteó una arquitectura híbrida desarrollada con Azure de la siguiente manera.

![Texto alternativo](../imgs/ArquitecturaActual.png)

Esta arquitectura posee un flujo de trabajo sobre el cual la ingesta de datos se vuelve más automatizada por medio de un pipeline en el servicio de Azure Data Factory integrado con Github, el cual recibe un archivo JSON y lo convierte dependiendo de la necesidad en un CSV o un Parquet, esto es introducido como parámetro al momento de correr el pipeline, esto permite una escalabilidad de la solución a futuro.

![Texto alternativo](../imgs/DataFactory.png)

Los datos son almacenados también en la nube posterior a esto en un Data Lake Storage Gen 2, un sistema de almacenamiento muy eficiente de Azure el cual permite bastantes tipos de datos con una gran eficiencia de lectura y escritura. Allí la data se separa en tres contenedores sobre los cuales el pipeline irá moviendo la información.

![Texto alternativo](../imgs/DLSGen2.png)

- Bussines: La fuente de datos cruda.
- Landing: Los datos listos para ser procesados, en esta etapa ya el archivo JSON pasa a ser Parquet. (Ver mapeo de variables en la imagen posterior)
- Work: Los datos resultado de las preguntas planteadas en el challenge.

![Texto alternativo](../imgs/MapeoVariables.png)

Finalmente, los datos son procesos en Python de forma local, donde se da respuesta a la solución. Se intentó realizar la solución sobre Databricks, sin embargo, la conexión no fue posible entre Data Lake Storage Gen 2 y Databrick por limitaciones de la suscripción que se posee. Era necesaria una licencia Pay as you go. Debido a esto se plantea una arquitectura similar donde se emplee por completo una integración en nube pública, es la siguiente, donde incluso se llega al punto de generar una visualización de la información por medio de Power BI, y pasando por una zona de validación de datos (Curated), antes de poder ser consumida ya sea por un desarrollo de BA o BI.

![Texto alternativo](../imgs/ArquitecturaPropuesta.png)

## Challenge
Tener en cuenta que para realizar estas soluciones se trabaja con un archivo parquet creado a partir del mapeo de lo observado en el archivo inicial formato JSON, manteniendo la cantidad total de la información intacta.

In [31]:
# Importe de librerias
import pandas as pd
from time import time

# Se define ruta donde se encuentra la data del contenedor landing
data_path = '../data/data.parquet'
ansuwers_path = '../data/answers'

### q1_memory
En este ejercicio se debe mostrar el top 10 de fechas donde más tweets fueron realizados, con su respectiva cantidad optimizando la memoria. 
Para primero es importante recalcar que desde la ingesta y trasformación de los datos desde Data Factory fue posible disminuir considerablemente el tamaño del archivo, pasando de un JSON de 398MB a un Parquet de 73MB, posterior a esto, el código optimiza la memoria cargando desde un inicio solo la columna con la cual se va a trabajar además de buscar usar la mínima cantidad posible de variables.

In [44]:
from q1_memory import q1_memory

# solución
initial_time = time()
result_q1 = q1_memory(data_path)
print(f"Tiempo: {time()-initial_time} seg")

result_q1

Tiempo: 0.18770051002502441 seg


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

### q1_time
En este ejercicio se debe mostrar el top 10 de fechas donde más tweets fueron realizados, con su respectiva cantidad optimizando el tiempo. 
Para este ejercicio se plantea una lectura solo de las columnas necesarias de la data, además de eliminar los ciclos for del código.

In [38]:
from q1_time import q1_time

# solución
initial_time = time()
result_q1 = q1_time(data_path)
print(f"Tiempo: {time()-initial_time} seg")

result_q1

Tiempo: 0.1103358268737793 seg


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [43]:
# Se guarda la respuesta
df1 = pd.DataFrame(result_q1, columns=['Fecha', 'Cantidad'])
df1.to_csv(f"{ansuwers_path}/AnswerQ1.csv", index=False)

### q2_memory
En este ejercicio se debe mostrar el top 10 de los emojis más usados de los contenidos de los tweets con su respectiva cantidad, optimizando la memoria. 
Para esto se disminuyó la cantidad de variables que se almacenen en memoria de forma significativa, este reto tiene una particularidad interesante comparado con el anterior el cual fue uno de los retos a nivel personal importantes aunque parezca simple, el hecho de disminuir la memoria teniendo en cuenta que es necesario realizar bucles sobre la tabla y además sobre el texto.

In [45]:
from q2_memory import q2_memory

# solución
initial_time = time()
result_q2 = q2_memory(data_path)
print(f"Tiempo: {time()-initial_time} seg")

result_q2

Tiempo: 4.053999662399292 seg


[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('|', 2429),
 ('✊', 2411),
 ('🌾', 2363),
 ('🇮', 2096),
 ('🇳', 2094),
 ('❤', 1779),
 ('🤣', 1668)]

### q2_time
En este ejercicio se debe mostrar el top 10 de los emojis más usados de los contenidos de los tweets con su respectiva cantidad, optimizando el tiempo. 
Al igual que el primer ejercicio, se busca disminuir la cantidad de ciclos en el proceso, además de segmentar un poco el código, separando la función de validación de si el carácter es un emoji o no. 

In [46]:
from q2_time import q2_time

# solución
initial_time = time()
result_q2 = q2_time(data_path)
print(f"Tiempo: {time()-initial_time} seg")

result_q2

Tiempo: 4.150782108306885 seg


[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('|', 2429),
 ('✊', 2411),
 ('🌾', 2363),
 ('🇮', 2096),
 ('🇳', 2094),
 ('❤', 1779),
 ('🤣', 1668)]

In [47]:
# Se guarda la respuesta
df2 = pd.DataFrame(result_q1, columns=['Emoji', 'Cantidad'])
df2.to_csv(f"{ansuwers_path}/AnswerQ2.csv", index=False)

### q3_memory
En este ejercicio se debe mostrar el top 10 de los usuarios mas mencionados en los tweets con su respectiva cantidad, optimizando la memoria. 
Se realiza un planteamiento igual a los mencionados en ejercicios anteriores, tengiendo en cuenta que la data al emplear un archivo parquet se va a optimizar bastante, la dantidad de datos no es muy grande para percibir cambios significativos en el uso de la memoria, además de poseer un script mas sencillo que los ejercicios apsados.

In [48]:
from q3_memory import q3_memory

# solución
initial_time = time()
result_q3 = q3_memory(data_path)
print(f"Tiempo: {time()-initial_time} seg")

result_q3

Tiempo: 0.038260459899902344 seg


[('narendramodi', 1135),
 ('Kisanektamorcha', 985),
 ('RakeshTikaitBKU', 782),
 ('PMOIndia', 674),
 ('RaviSinghKA', 584),
 ('YouTube', 568),
 ('Tractor2twitr', 493),
 ('RahulGandhi', 460),
 ('DelhiPolice', 369),
 ('rihanna', 364)]

### q3_time
En este ejercicio se debe mostrar el top 10 de los usuarios mas mencionados en los tweets con su respectiva cantidad, optimizando el tiempo. 
Se busca disminuir al maximo los bucles repetitivos sobre el dataframe, además de poseer una variable propia, diccionario para el almacenamiento de resultados.

In [49]:
from q3_time import q3_time

# solución
initial_time = time()
result_q3 = q3_time(data_path)
print(f"Tiempo: {time()-initial_time} seg")

result_q3

Tiempo: 0.022534847259521484 seg


[('narendramodi', 1135),
 ('Kisanektamorcha', 985),
 ('RakeshTikaitBKU', 782),
 ('PMOIndia', 674),
 ('RaviSinghKA', 584),
 ('YouTube', 568),
 ('Tractor2twitr', 493),
 ('RahulGandhi', 460),
 ('DelhiPolice', 369),
 ('rihanna', 364)]

In [ ]:
# Se guarda la respuesta
df3 = pd.DataFrame(result_q1, columns=['Usuario', 'Cantidad'])
df3.to_csv(f"{ansuwers_path}/AnswerQ3.csv", index=False)